# Generate Data

This notebook generates batches of simulated MRS spectra using the MRS phantom framework.
It performs the following steps:

1. Load a configuration file containing simulation parameters.
2. Split the main configuration into individual configurations for each simulation instance.
3. Run simulations for each configuration using the MRS phantom.
4. Save the results (e.g., spectra, ground truth data) for further processing or validation.

> **Note:** Use this notebook when you want to generate a new dataset of simulated MRS spectra in a reproducible and configurable way.


## Imports

In [ ]:
# === Library Imports ===
import json                                         # For loading and parsing configuration files
import sys                                          # For system-specific parameters and functions
from tqdm import tqdm                               # Progress bar for long-running loops
import os                                           # For file and directory operations
from PyQt5.QtWidgets import QApplication            # GUI backend required for voxel selector

# === Set up the project root directory ===
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(PROJECT_ROOT)

# === Local Simulation Core ===
from simulation.mrs_phantom import DigitalPhantom   # Core class for generating phantom-based MRS data
from simulation.run_matlab import MatlabRunner      # Interface to run MATLAB components (if needed)

# === Local GUI Tools ===
from gui.main_voxel_selector import PhantomWindow   # GUI for interactive voxel selection
from gui.defaults import *                          # Default GUI settings (window size, paths, etc.)

# === Local Utility Functions ===
from utils.auxillary import (
    split_config_into_simulations,                  # Break main config into per-simulation configs
    group_simulation_configs_by_settings,           # Group similar configs for efficient processing
    unflatten_dict                                  # Reconstruct nested dictionaries from flattened ones
)
from utils.nii_processing import save_nifti_mrs     # Save MRS data in NIfTI-MRS format

## Settings

In [ ]:
# === Load initial configuration ===

# Path to the main configuration file
config_path = './config.json'

# Flag to enable or disable manual voxel selection via GUI
select_voxel = True

# Load the configuration from the JSON file
with open(config_path, 'r') as file:
    config = json.load(file)


## Optional: Launch Voxel Selection GUI

In [ ]:
# === Optional: Launch voxel selection GUI ===
# Initialize a DigitalPhantom object using the skeleton provided in the config
# This is only needed when GUI-based voxel selection is enabled

if select_voxel:
    phantom = DigitalPhantom(
        skeleton=config['skeleton'],               # Tissue label + anatomical information
        path2metabs=DEFAULT_METAB_DF_PATH,         # Default metabolite database
        path2basis=DEFAULT_path2basis,             # Default basis spectra path
        simulation_params=DEFAULT_SIM_PARAMS,      # Default simulation parameters
    )

    # Initialize Qt application and show the Phantom GUI window
    app = QApplication([])                         # Create an empty QApplication (required for GUI)
    window = PhantomWindow(phantom, config, output_dir='outputs')
    window.show()                                  # Launch the GUI
    app.exec_()                                    # Enter the event loop until the GUI is closed

    # Update the config path with the file generated by the GUI
    config_path = window.config_path


# Run Simulations from Config Files
This block runs the batch simulation process. It:

1. Splits the main config into per-voxel simulation configs.
2. Groups those configs by simulation settings (to reuse basis sets).
3. Checks or generates required basis sets.
4. Initializes the DigitalPhantom.
5. Simulates MRS data for each voxel and saves spectra as NIfTI-MRS files.

In [ ]:
# === Step 1: Split main config into individual simulation configs ===
split_config_into_simulations(config_path)
print("Simulation configurations generated successfully.")

# === Step 2: Group configs by shared simulation/basis settings ===
groups = group_simulation_configs_by_settings(config_path)

idx = 0  # Spectrum index counter for naming
# === Step 3: Loop over each group of settings ===
for settings_key, config_files in tqdm(groups.items(), desc="Processing settings groups", unit="group"):
    
    settings = unflatten_dict(settings_key)
    print(f"\nInitializing simulation with settings: {settings_key}")

    # === Step 4: Determine basis set path ===
    if 'path2basis' in settings:
        # Use provided basis set path
        basis_path = settings['path2basis']
        
        if os.path.exists(basis_path):
            print(f"Using provided basis set: {basis_path}")
        else:
            print(f"Provided basis set path does not exist: {basis_path}")
            print("Falling back to default basis set generation...")

            # Compose expected basis filename
            expected_file_name = f"LCModel_{settings['basis_set_settings']['vendor']}_UnEdited_" \
                                 f"{settings['basis_set_settings']['localization']}_TE" \
                                 f"{settings['basis_set_settings']['TE']}.BASIS"
            basis_path = os.path.join(settings['basis_set_dir'], expected_file_name)

            if os.path.exists(basis_path):
                print(f"Found fallback basis set: {basis_path}")
            else:
                print(f"Generating basis set: {expected_file_name}")
                matlab_runner = MatlabRunner()
                matlab_runner.generate_basis_set(settings['basis_set_settings'])
                print(f"Basis set generated: {basis_path}")

    else:
        # No basis path provided, generate expected filename and check
        expected_file_name = f"LCModel_{settings['basis_set_settings']['vendor']}_UnEdited_" \
                             f"{settings['basis_set_settings']['localization']}_TE" \
                             f"{settings['basis_set_settings']['TE']}.BASIS"
        basis_path = os.path.join(settings['basis_set_dir'], expected_file_name)

        if os.path.exists(basis_path):
            print(f"Using existing basis set: {basis_path}")
        else:
            print(f"Basis set not found: {basis_path}")
            print("Generating new basis set...")
            matlab_runner = MatlabRunner()
            matlab_runner.generate_basis_set(settings['basis_set_settings'])
            print(f"Basis set generated: {basis_path}")

    # === Step 5: Combine simulation and basis parameters ===
    sim_parms = settings['simulation_params']
    basis_params = settings['basis_set_settings']
    all_params = {**sim_parms, **basis_params}

    # === Step 6: Initialize phantom with current settings ===
    phantom = DigitalPhantom(
        skeleton=settings['skeleton'],
        path2metabs=settings['path2metabs'],
        path2basis=basis_path,
        simulation_params=all_params,
        gui=None,  # GUI not used during batch simulations
        metabs=settings['metabs'],
    )

    # === Step 7: Set save directory ===
    save_dir = os.path.join(os.path.dirname(config_path), 'spectra')
    os.makedirs(save_dir, exist_ok=True)

    # === Step 8: Loop over each voxel config in the group ===
    for file in tqdm(config_files, desc=f"Simulating voxels for group {idx}", unit="voxel", leave=False):
        with open(file, 'r') as f:
            config = json.load(f)

        voi_coords = config['voxel_definitions']['coords']

        # Run the simulation
        spec, components, t, ppm = phantom.simulate_data(voi_coords)

        # === Step 9: Save result as NIfTI-MRS file ===
        save_nifti_mrs(
            save_dir,
            total_selection=["Metabs", "MMs", "Lipids", "Water", "Noise"],
            individual_selection=["Metabs", "MMs", "Lipids", "Water", "Noise"],
            components_data=components,
            affine=phantom.affine,
            dwelltime=phantom.basis_set.dwelltime,
            spec_freq=phantom.basis_set.cf,
            sim_params=config,
            base_name=f"simulated_spectrum_{idx}",
        )

        idx += 1

        # Clean up temporary config file
        os.remove(file)

print("✅ All simulations completed and spectra saved!")
